In [30]:
import numpy as np
import gmsh


gmsh.initialize()
gmsh.model.add('PROBLEMA1')


# Generado de la figura
L1 = 20
L2 = 10
#Lc = 20 #caso del ejercicio con 4 triangulos
Lc = 1

p1 = gmsh.model.geo.addPoint(0, 0, 0, Lc)
p2 = gmsh.model.geo.addPoint(0, L2, 0, Lc) 
p3 = gmsh.model.geo.addPoint(L1, L2, 0, Lc) 
p4 = gmsh.model.geo.addPoint(L1, 0, 0, Lc)

l1 = gmsh.model.geo.addLine(p1, p2)
l2 = gmsh.model.geo.addLine(p2, p3)
l3 = gmsh.model.geo.addLine(p3, p4)
l4 = gmsh.model.geo.addLine(p4, p1)

C1 = gmsh.model.geo.addCurveLoop([l1, l2, l3, l4])

S1 = gmsh.model.geo.addPlaneSurface([C1])




#Generado de las condiciones de contorno

Empotrado = gmsh.model.addPhysicalGroup(1, [l1]) #1 es para curva o linea!
gmsh.model.setPhysicalName(1,Empotrado,'Empotrado')

Traccionado = gmsh.model.addPhysicalGroup(1, [l3])
gmsh.model.setPhysicalName(1,Traccionado,'Traccionado') #esta funcion le pone nombre a todos los nodos aca

Superficie = gmsh.model.addPhysicalGroup(2,[S1]) #2 es para superficies!
gmsh.model.setPhysicalName(2,Superficie, 'Superficie')


gmsh.model.geo.synchronize() #sincroniza lo que creamos con el modelo que creamos arriba
gmsh.model.mesh.generate(2)

#gmsh.fltk.run()

In [31]:
NodeInfo = gmsh.model.mesh.get_nodes()
NodeInfo[1].shape
NumeroNodos = NodeInfo[0].shape[0]
MN = NodeInfo[1].reshape(NumeroNodos , 3)


ElementInfo = gmsh.model.mesh.get_elements()
ETAGS, ELEMENTS = gmsh.model.mesh.get_elements_by_type(2)
MC = ELEMENTS.reshape([ETAGS.shape[0],3])


In [32]:
#Empotrados
NodosEmpotrados = gmsh.model.mesh.get_nodes_for_physical_group(1,Empotrado)
A = NodosEmpotrados[0].reshape(-1,1)
B = np.zeros((len(A),2))
MNE = np.concatenate((A,B) , axis = 1)

#Traccionados
NodosTraccionados = gmsh.model.mesh.get_nodes_for_physical_group(1, Traccionado)
C = NodosTraccionados[0].reshape(-1,1)
D = np.zeros((len(C),2))
MNT = np.concatenate((C,D) , axis = 1)
MNT[:,1] = 5000



#resto?

In [33]:
Nodoslibres = []

Nodos = gmsh.model.mesh.get_nodes_for_physical_group(2, Superficie)
for i in Nodos[0]:
    if i in NodosTraccionados[0]:
        l = 0
    elif i in NodosEmpotrados[0]:
        l = 0
    else:
        Nodoslibres.append(i)
        
E = np.array(Nodoslibres).reshape(-1,1)
D = np.zeros((len(E),2))
Interior = np.concatenate((E,D) , axis = 1)
MNT = np.concatenate((Interior,MNT) , axis = 0)

gmsh.finalize()

In [34]:
#MNT[np.argsort(MNT[:,0])]


In [35]:
print(MN)
print()
print(MC)
print()
print(MNE)
print()
print(MNT)

[[ 0.          0.          0.        ]
 [ 0.         10.          0.        ]
 [20.         10.          0.        ]
 [20.          0.          0.        ]
 [ 0.          1.          0.        ]
 [ 0.          2.          0.        ]
 [ 0.          3.          0.        ]
 [ 0.          4.          0.        ]
 [ 0.          5.          0.        ]
 [ 0.          6.          0.        ]
 [ 0.          7.          0.        ]
 [ 0.          8.          0.        ]
 [ 0.          9.          0.        ]
 [ 1.         10.          0.        ]
 [ 2.         10.          0.        ]
 [ 3.         10.          0.        ]
 [ 4.         10.          0.        ]
 [ 5.         10.          0.        ]
 [ 6.         10.          0.        ]
 [ 7.         10.          0.        ]
 [ 8.         10.          0.        ]
 [ 9.         10.          0.        ]
 [10.         10.          0.        ]
 [11.         10.          0.        ]
 [12.         10.          0.        ]
 [13.         10.        

In [36]:
espesor = 1
E = 30E6
v = 0.3

In [37]:
D = (E/(1-v**2))*np.array([[1,v,0],[v,1,0],[0,0,(0.5*(1-v))]])
Matriz_coordenadas = np.ones((3,3))
Matriz_global = np.zeros([2*len(MN),2*len(MN)])
Matrices_Bloc = []

In [38]:
for i in range (len(MC)):
    
    I,M,J = MC[i]    
    i = int(I - 1) ; m = int(M - 1) ; j = int(J - 1)
    
    x = np.array([MN[i,0],MN[m,0],MN[j,0]])
    y = np.array([MN[i,1],MN[m,1],MN[j,1]])
    Bm = y[0] - y[2] ;   Bi = y[2] - y[1] ;    Bj = y[1] - y[0]
    Gi = x[1] - x[2] ;   Gm = x[2] - x[0] ;    Gj = x[0] - x[1]  
    
    Matriz_coordenadas[:,0] = x
    Matriz_coordenadas[:,1] = y
    
    Area = abs(np.linalg.det(Matriz_coordenadas)/2)

    Matriz1 = (1/(2*Area)) * np.array([[Bi,0,Bj,0,Bm,0],[0,Gi,0,Gj,0,Gm],[Gi,Bi,Gj,Bj,Gm,Bm]])

    Matrices_Bloc.append(Matriz1)
    Matriz_local = espesor * Area * (np.dot(np.transpose(Matriz1), np.dot(D,Matriz1)))

    A_G = np.array([i*2,i*2+1,j*2,j*2+1,m*2,m*2+1]) #aca pasa algo raro
    Matriz_global[np.ix_(A_G,A_G)] += Matriz_local
    
print(Matriz_global*(0.91/375000))

[[ 44.82685902  13.93335801   0.         ...   0.           0.
    0.        ]
 [ 13.93335801  44.82685902   0.         ...   0.           0.
    0.        ]
 [  0.           0.          44.82685902 ...   0.           0.
    0.        ]
 ...
 [  0.           0.           0.         ... 186.6376051    0.
    0.        ]
 [  0.           0.           0.         ...   0.         193.10166377
  -19.50970354]
 [  0.           0.           0.         ...   0.         -19.50970354
  210.93209143]]


In [39]:
Desplazamientos_f = np.zeros(len(MN)*2)
Fuerza = []
Nodos = []


for i in MNE:
    Nodo,valorx,valory = i 
    nodo = int(Nodo-1)
    Desplazamientos_f[2*nodo] = valorx
    Desplazamientos_f[2*nodo+1] = valory


for i in MNT:
    Nodo,valorx,valory = i
    nodo = int(Nodo-1)
    Fuerza.append(valorx)
    Fuerza.append(valory)
    Nodos.append(2*nodo)
    Nodos.append(2*nodo+1) 


Fuerza = np.array(Fuerza).reshape(-1,1)

Matriz_res = Matriz_global[np.ix_(Nodos,Nodos)]

Desplazamientos_parcial = np.linalg.solve(Matriz_res,Fuerza)


o = 0
for i in Nodos:
    Desplazamientos_f[i] = Desplazamientos_parcial [o]
    o += 1
  

Fuerzas_ = np.dot(Matriz_global,Desplazamientos_f)
print(Fuerzas_.round(4))
print()
print(Desplazamientos_f*1e6)

[-3.8327082e+03 -1.2664490e+03 -3.8386384e+03  1.2745646e+03
  5.0000000e+03 -0.0000000e+00  5.0000000e+03  0.0000000e+00
 -5.5500247e+03 -8.8041690e+02 -5.2459960e+03 -5.3886950e+02
 -5.1650516e+03 -3.7954600e+02 -5.1393393e+03 -1.8195250e+02
 -5.1338509e+03  9.6850000e-01 -5.1405273e+03  1.7671420e+02
 -5.1680721e+03  3.6172370e+02 -5.2419599e+03  5.7493470e+02
 -5.5438317e+03  8.5832830e+02  0.0000000e+00  0.0000000e+00
  0.0000000e+00 -0.0000000e+00  0.0000000e+00 -0.0000000e+00
  0.0000000e+00 -0.0000000e+00 -0.0000000e+00  0.0000000e+00
 -0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
 -0.0000000e+00  0.0000000e+00  0.0000000e+00 -0.0000000e+00
 -0.0000000e+00  0.0000000e+00  0.0000000e+00 -0.0000000e+00
  0.0000000e+00  0.0000000e+00 -0.0000000e+00 -0.0000000e+00
  0.0000000e+00  0.0000000e+00  0.0000000e+00 -0.0000000e+00
  0.0000000e+00 -0.0000000e+00 -0.0000000e+00  0.0000000e+00
 -0.0000000e+00  0.0000000e+00  0.0000000e+00 -0.0000000e+00
  5.0000000e+03  0.00000

In [40]:
#Esfuerzos

Tensiones = []

for i in range(len(MC)):
    
    I,M,J = MC[i]    
    i = int(I - 1) ; m = int(M - 1) ; j = int(J - 1)

    
    nodos = [2*i,2*i+1,2*j,2*j+1,2*m,2*m+1]
    
    
    A = np.dot(Matrices_Bloc[i],Desplazamientos_f[nodos])
    
    B = np.dot(D,A)
    
   
    Tensiones.append(B)

for i in range (len(Tensiones)):
    print(Tensiones[i])
    print()

[ -692.78185262  1894.1613194  -2715.70451484]

[5811.09874571 -276.04430539 -116.73093057]

[ 1398.45105359   744.36823778 -2879.72448825]

[-4853.52854693  -264.14070173   246.55344447]

[  446.03695407   330.89840132 -2889.65087417]

[5377.56061883 -267.252998   -121.86183099]

[-5036.64226783   103.09410368  -333.04476201]

[-5807.79152801  -833.52145507  1214.14102964]

[-5988.18469641  -127.9203591     21.35512114]

[4570.69229304 -217.78772085 1515.47751802]

[ 2477.02706589   232.76204844 -2790.54368992]

[-2855.46550539 -1744.8683451   2331.79087181]

[5566.12474387   82.72898922 1704.90409377]

[ 2588.21832812  -542.41320539 -2376.72593572]

[ 5181.48397871   -35.59113568 -1538.20017093]

[ 5834.24498391   180.44321918 -1042.18656197]

[ 5034.36872601   105.76806018 -1345.6077724 ]

[-2120.77535938   226.13335688  2594.69389746]

[ 5441.89601572   619.43116496 -1504.6213333 ]

[-5407.143628    -276.57123921   628.42338197]

[ 3196.91780832   183.43373236 -1926.82054146]

[-53